In [1]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
url = "http://projects.fivethirtyeight.com/soccer-api/club/spi_matches.csv"

In [4]:
df = pd.read_csv(url).dropna()
df = df[(df.league=="Barclays Premier League")|(df.league=="German Bundesliga")]

In [5]:
res = []

In [6]:
df["h_margin"] = df["score1"] - df["score2"]

In [7]:
for x in df["h_margin"]:
    if x > 0:
        res.append("HOME WIN")
    elif x == 0:
        res.append("DRAW")
    else:
        res.append("AWAY WIN")

In [8]:
df["result"] = res

In [9]:
X_final = df[["score1","score2","h_margin"]]
y_final = df[["result"]].copy()
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2, random_state=6)

In [11]:
def categorize(array):
    array = tf.keras.utils.to_categorical(np.asarray(pd.factorize(array)[0]))
    return array

In [12]:
y_train = categorize(y_train["result"])
y_test = categorize(y_test["result"])

In [16]:
def load_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(16, activation="relu"))
    model.add(tf.keras.layers.Dense(3, activation="softmax"))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [17]:
model = load_model()

In [18]:
model.fit(X_train, y_train, epochs=25, verbose=1, batch_size=128, validation_split=0.2)

Epoch 1/25
17/17 [==============================] - 1s 17ms/step - loss: 0.8649 - accuracy: 0.5511 - val_loss: 0.7766 - val_accuracy: 0.6667
Epoch 2/25
17/17 [==============================] - 0s 4ms/step - loss: 0.7058 - accuracy: 0.6828 - val_loss: 0.6464 - val_accuracy: 0.7341
Epoch 3/25
17/17 [==============================] - 0s 4ms/step - loss: 0.6012 - accuracy: 0.7807 - val_loss: 0.5602 - val_accuracy: 0.7940
Epoch 4/25
17/17 [==============================] - 0s 3ms/step - loss: 0.5278 - accuracy: 0.8121 - val_loss: 0.4994 - val_accuracy: 0.8184
Epoch 5/25
17/17 [==============================] - 0s 3ms/step - loss: 0.4735 - accuracy: 0.8538 - val_loss: 0.4534 - val_accuracy: 0.9363
Epoch 6/25
17/17 [==============================] - 0s 3ms/step - loss: 0.4316 - accuracy: 0.9414 - val_loss: 0.4165 - val_accuracy: 0.9363
Epoch 7/25
17/17 [==============================] - 0s 3ms/step - loss: 0.3972 - accuracy: 0.9419 - val_loss: 0.3854 - val_accuracy: 0.9363
Epoch 8/25
17/17 [=

In [19]:
predictions = model.predict(X_test)

In [20]:
loss, accuracy = model.evaluate(y_test, predictions)

21/21 [==============================] - 0s 2ms/step - loss: 1.1981 - accuracy: 0.4408


In [21]:
print("Test Accuracy: {}%".format(accuracy * 100))

Test Accuracy: 44.07795965671539%
